In [1]:
from interfacemaster.cellcalc import get_primitive_hkl
from interfacemaster.interface_generator import core, print_near_axis, convert_vector_index
from numpy import array, dot, round, var, average
from numpy.linalg import inv
from gb_code.csl_generator import print_list

# Define an interface core class

Input two cif files.For GBs, input the same file; for interfaces, input two different files.


The first structure is non-changed while the second one can be deformed to form an approximate CSL

Please input files of CONVENTIONAL cells if you are not familiar with the indices in the primitive cell.

In [2]:
my_interface = core('cif_files/Si_mp-149_conventional_standard.cif','cif_files/Si_mp-149_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


# Specify the rotation axis and searching parameters to find an approximate CSL

Since this is for a cubic lattice, we have determined rotation axis/angle to give a specific CSL

In [3]:
print_list([1,1,1],100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  60.00 
Sigma:     7  Theta:  38.21 
Sigma:    13  Theta:  27.80 
Sigma:    19  Theta:  46.83 
Sigma:    21  Theta:  21.79 
Sigma:    31  Theta:  17.90 
Sigma:    37  Theta:  50.57 
Sigma:    39  Theta:  32.20 
Sigma:    43  Theta:  15.18 
Sigma:    49  Theta:  43.57 
Sigma:    57  Theta:  13.17 
Sigma:    61  Theta:  52.66 
Sigma:    67  Theta:  24.43 
Sigma:    73  Theta:  11.64 
Sigma:    79  Theta:  33.99 
Sigma:    91  Theta:  10.42 
Sigma:    93  Theta:  42.10 
Sigma:    97  Theta:  30.59 


Rotation axis should be expressed in the primitive cell frame, we can convert this.

For example we need an axis [1,0,0] expressed in the conventional cell.

In [12]:
axis = convert_vector_index(my_interface.conv_lattice_1, my_interface.lattice_1, [1,0,0])
axis

array([-1., -1.,  1.])

For the meaning of searching parameters, please check the paper.

If you are making an exact CSL, just set du, S & dd to be 'very small'. 

In this case we make a known twinning GB, which is an exact CSL.

In [13]:
my_interface.parse_limit(du = 1e-5, S  = 1e-5, sgm1=100, sgm2=100, dd = 1e-5)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(axis,36.87,1,0.01)

[5.4687280e+00 0.0000000e+00 4.4408921e-16]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1  2]
 [ 1  0 -1]
 [-1  2  0]]; sigma_1 = 5

U2 = 
[[ 1  0  2]
 [ 1 -1 -1]
 [-1  2 -1]]; sigma_2 = 5

D = 
[[ 1.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  1.00e+00  1.79e-06]
 [-0.00e+00 -1.79e-06  1.00e+00]]

axis = [5.4687280e+00 0.0000000e+00 4.4408921e-16] ; theta = 36.87



'U1', 'U2' are two sets of indices of the CSL in the two crystals' frames,
'D' is the deformation applied to the crystal 2 to form an approximate CSL

As you can see, to form a twinning structure we need to deform the structure provided!

# Convert the miller indices to be expressed in the primitive cell

Now we need to specify the orientation of our interface, which is by giving a miller indices in expressed in the primitive cell.

In [14]:
"""
The interface plane is the [0,-1,3] plane in the conventional cell and now we \
transform it into primitive cell
"""
hkl = get_primitive_hkl(array([0,-1,-2]), my_interface.conv_lattice_1, my_interface.lattice_1)
hkl

array([1, 2, 3])

# Compute the indices of the two slabs to make bicrystal

Now let's compute the indices of the two slabs forming the interface
If you want a near orthogonal bicrystal, please set 'orthogonal to yes, 
and adjust the lim & tol.

In [15]:
my_interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-2)

cell 1:
[[ 1 -1 -3]
 [-1 -1  3]
 [-3  1 -1]]
cell 2:
[[ 1 -1 -3]
 [-1 -1  3]
 [-3  1 -1]]


Here we obtained the two sets of indices to build a bicrystal

# Make a bicrystal!

You can adjust the bicrystal by
1. do expansion by set 'xyz' = [dimX, dimY, dimZ]
2. do RBT by 'dzdy' = a translation vector 
(please confine dzdy in the interface plane, 
for RBT in the normal, please use 'dp1' and 'dp2', to make vacuum in the interface use 'dx');
3. introduce vacuum in the interface by 'dx' = the distance spacing the two crystals
4. adjust the termination by 'dp1', 'dp2' = length of shift of the position of 
the termination from the interface into the bulk
5. introduce vacuum to make a surface in the end of the bicrystal by
'vx' = length of the vaccumn

default: xyz = [1,1,1] and others = 0

In [18]:
my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1], filetype='VASP')

[[ 1.22284476e+01 -2.24632937e-16 -8.61053934e-16]
 [ 4.06386332e-32  5.46872800e+00  2.76657704e-16]
 [-3.47670651e-16  0.00000000e+00  1.22284476e+01]]


Good! You have made a 'POSCAR' file of this bicrystal, we also generated two files named 'cell_1.cif' and 'cell_2.cif' providing the structure of the two slabs.

In [13]:
#To sample the CNID, try
my_interface.sample_CNID(grid = [4,4], xyz_1 = [1,1,1], xyz_2 = [1,1,1])

CNID
[[ 1.  0.]
 [ 1.  1.]
 [-1.  1.]]
making 400 files...
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01

[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.4687

[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.4687

[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.4687

[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.4687

[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.46872800e+00  5.02294518e-16]
 [-6.34275259e-16  0.00000000e+00  1.22284476e+01]]
[[ 1.22284476e+01  1.23725086e-16  9.54821425e-16]
 [ 2.33418319e-32  5.4687

Please check the generated 'POSCAR.x.y' files. 
The CNID here is expressed in the primitive cell's frame.
You can refer to the cartesian coordinates or in the conventional cell's frame

In [10]:
CNID_cartesian = round(my_interface.CNID,8)
CNID_cartesian

array([[-5.468728, -2.734364],
       [ 0.      , -2.734364],
       [ 0.      , -5.468728]])

In [11]:
CNID_conv = round(dot(inv(my_interface.conv_lattice_1)\
                              , my_interface.CNID),8)
CNID_conv

array([[-1. , -0.5],
       [ 0. , -0.5],
       [ 0. , -1. ]])